## Задание №2

### -2. Установить библиотеку для соединения с СУБД

In [10]:
pip install psycopg2-binary pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### -1. Установить соединение с СУБД

In [11]:
import psycopg2 as ps
import pandas as pd

def exec_sql(p_sql):
    # Fetch variables
    USER = 'postgres.oqwkbszzvzemgqyyebuw'
    PASSWORD = '8FCX8HZ2r1PBYBJy'
    HOST = 'aws-1-eu-west-1.pooler.supabase.com'
    PORT = 6543
    DBNAME = 'postgres'

    # Connect to the database
    try:
        connection = ps.connect(
            user=USER,
            password=PASSWORD,
            host=HOST,
            port=PORT,
            dbname=DBNAME
        )
        print("Connection successful!")

        # Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Example query
        cursor.execute(p_sql)
        draw_result(cursor)

        # Close the cursor and connection
        cursor.close()
        connection.close()
        print("Connection closed.")

    except Exception as e:
        print(f"Failed to connect: {e}")


def draw_result(cursor):
    result = cursor.fetchall()
    fields = [i[0] for i in cursor.description]
    rows = [list(i) for i in result]
    df = pd.DataFrame(rows, columns=fields)
    display(df)


def query_get_sysdate():
    return "SELECT NOW() \"Текущее время\";"

exec_sql(query_get_sysdate())

Connection successful!


,Текущее время
0,2025-11-18 11:55:44.547432+00:00


Connection closed.


### 0. Добавить таблицу insurance_final_rows в supabase
Таблица импортирована, проверяем её наличие и данные в ней
```sql
SELECT *
  FROM insurance_final_rows
 LIMIT 5;
```

In [12]:
def query_get_insurance_lim5():
    return ("SELECT * "
            "  FROM insurance_final_rows "
            " LIMIT 5;")

exec_sql(query_get_insurance_lim5())

Connection successful!


,user_id,gender,age,salary,family_members,insurance_appeal
0,1,True,41,49600.0,1,0
1,2,False,46,38000.0,1,1
2,3,False,29,21000.0,0,0
3,4,False,21,41700.0,2,0
4,5,True,28,26100.0,0,0


Connection closed.


### 1. Вывести всех пользователей, которые имеют больше одного члена семьи.
```sql
SELECT *
  FROM insurance_final_rows
 WHERE family_members > 1;
```

In [13]:
def query_get_insurance_more_one_family_mmbr():
    return ("SELECT * "
            "  FROM insurance_final_rows "
            " WHERE family_members > 1 "
            " LIMIT 5;")

exec_sql(query_get_insurance_more_one_family_mmbr())

Connection successful!


,user_id,gender,age,salary,family_members,insurance_appeal
0,4,False,21,41700.0,2,0
1,6,True,43,41000.0,2,1
2,7,True,39,39700.0,2,0
3,8,True,25,38600.0,4,0
4,13,False,23,39500.0,3,0


Connection closed.


### 2. Посчитать среднюю зарплату среди женщин.
```sql
SELECT round(avg(salary), 2) avg_salary_women
  FROM insurance_final_rows
 WHERE gender = false;
```

In [14]:
def query_get_insurance_avg_salary_women():
    return ("SELECT ROUND(AVG(salary)::NUMERIC, 2) \"Средняя зарплата среди женщин\" "
            "  FROM insurance_final_rows "
            " WHERE gender = FALSE;")

exec_sql(query_get_insurance_avg_salary_women())

Connection successful!


,Средняя зарплата среди женщин
0,39769.06


Connection closed.


### 3. Найти возраст самого молодого пользователя, который обращался за страховой выплатой хотя бы один раз.
```sql
SELECT MIN(age) min_age_user
  FROM insurance_final_rows
 WHERE insurance_appeal > 0;
```

In [15]:
def query_get_age_insuranced_youngest_user():
    return ("SELECT MIN(age) \"Возраст самого молодого пользователя\" "
            "  FROM insurance_final_rows "
            " WHERE insurance_appeal > 0;")

exec_sql(query_get_age_insuranced_youngest_user())

Connection successful!


,Возраст самого молодого пользоват
0,42


Connection closed.


### 4. Найти пользователя с максимальным количеством обращений за страховой выплатой и вывести его user_id, возраст и количество обращений.
```sql
SELECT
    user_id,
    age "Возраст",
    insurance_appeal "Количество обращений"
  FROM insurance_final_rows
 WHERE insurance_appeal = (SELECT MAX(insurance_appeal)
                             FROM insurance_final_rows);
```

In [16]:
def query_get_user_max_appeal():
    return ("SELECT "
            "   user_id, "
            "   age \"Возраст\", "
            "   insurance_appeal \"Количество обращений\" "
            "  FROM insurance_final_rows "
            " WHERE insurance_appeal = (SELECT MAX(insurance_appeal)"
            "                             FROM insurance_final_rows);")

exec_sql(query_get_user_max_appeal())

Connection successful!


,user_id,Возраст,Количество обращений
0,229,65,5


Connection closed.
